In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.optimize as opt
from functools import partial

# Structural Econometrics in Labor and IO.

# Problem Set: Dynamic Discrete Choice Estimation.

  
## Description

Superintendent Harry Zurcher manages the bus fleet at the Madison Metropolitan Bus Com- pany. Notably, at every point in time t, he decides whether to replace an old bus engine with a new one or to keep operating the old one.
We are interested in estimating the cost of engine maintenance and replacement, using observed data on: 

1. mileage $x_t$ at period $t$, 
2. observed replacement decision $i_t$ at period $t$, and
3. mileage $x_t+1$ next period.

#### The data generating process is as follows:

- $x_t$ can take on 11 values, $x_t \in \{0, 1, 2, . . . , 10\}$ (i.e. mileage discretized into 11 categories).

- At any time period, 
\begin{equation*}
x_{t+1} =
\begin{cases}
    \min\{x_t+1, \ 10\},& \text{ with probability } \lambda, \\
    x_t,& \text{ with probability } 1-\lambda.
\end{cases}
\end{equation*}
Note that $\lambda$ does not depend on the replacement decision $i_t$.
- The agent’s replacement decision is made at the beginning of each period and is effective immediately (i.e. if $i_t = 1$, the agent uses machine with mileage $0$ and the next period state $x_t+1$ is $1$ with probability $\lambda$ and $0$ with probability $1 − \lambda$).
- The per-period maintenance cost for a bus with mileage $x$ is $C(x, \theta) = \theta_1 x + \theta_2 x^2$.
- The cost of replacement is $\theta_3$.
- The per-period utility function is given by 
\begin{equation*}
u(x_t,i_t,\epsilon_{1t},\epsilon_{2t};\theta) =
\begin{cases}
    −\theta_1 x_t − \theta_2 x^2_t + \epsilon_{0t},& \text{ if } i_t = 0,\\
    −\theta_3 + \epsilon_{1t},& \text{ if } i_t = 1,
\end{cases}
\end{equation*}
where $(\epsilon_{0t}, \epsilon_{1t})$ ~ iid type 1 extreme value.
- Assume that the decision-maker's discount factor is known, $\beta = 0.95$.
- The number of observations is $N = 1000$.
- The true parameters are: $\theta_1 = 0.5$, $\theta_2 = 0.04$, $\theta_3 = 2$, and $\lambda = 0.82$.
- Hint: All busses are assumed to be operated and maintained independently.


In [2]:
# Import custom functions from python files
from generate_data import simulate_data
from dynamicprogramming import bellman1, bellman2
from estimation import rust_llf

In [3]:
seed = 100

Set parameters.

In [4]:
# number of observations
num_observations = 1000
# number of states
num_states = 11
# make vector of states
x = np.arange(0, num_states)

# set cost function parameters
theta = np.array([0.5, 0.04, 2])

params = {
    "beta": 0.95,
    "lambda": 0.82,
    "euler": 0.57721,
    "states": num_states,
    "x": x,
    "bellman_emax": 1,
    "threshold": 1e-12,
}

## Problem 1. Simulate data.

Solve dynamic programming problem to obtain replacement probabilities for each mileage state.

In [5]:
if params["bellman_emax"]:
    EV,V01,numiter = bellman1(
        theta,
        params,
)
else:
    EV,V01,numiter = bellman2(
    theta,
    params,
)

print(V01)

[[ -0.89603232  -2.89603232]
 [ -2.03912172  -2.89603232]
 [ -3.01029265  -2.89603232]
 [ -3.89832589  -2.89603232]
 [ -4.77152396  -2.89603232]
 [ -5.67421353  -2.89603232]
 [ -6.6322285   -2.89603232]
 [ -7.65922195  -2.89603232]
 [ -8.76171932  -2.89603232]
 [ -9.94254013  -2.89603232]
 [-11.2026467   -2.89603232]]


Simulate data for $N = 1000$ observations.

In [6]:
df = simulate_data(
    theta=theta, params=params, num_observations=num_observations, seed=seed
)
df

,xt,xt1,d
0,0,1,0
1,1,1,1
2,1,2,0
3,2,0,1
4,0,1,1
...,...,...,...
995,1,2,0
996,2,3,0
997,3,1,1
998,1,2,0


## Problem 2. Estimate the model using the simulated data.

Estimate and report transition probability λ in a first step. This does not require any nonlinear optimization.

In [7]:
# Estimating lambda - transition probabilities
# Frequency of x_t = x_t+1 = 10
numbound = np.sum((df["xt"] == 10) & (df["xt1"] == 10))
# Mileage up with d=0
numup_d0 = np.sum((df["d"] == 0) & (df["xt1"] - df["xt"] == 1))
# Mileage up with d=1
numup_d1 = np.sum((df["d"] == 1) & (df["xt1"] == 1))

# numbound=0; # If forgotten to account for boundary state
# Transition prob, net of boundary states
lmd = (numup_d0 + numup_d1) / (num_observations - numbound)

print("The estimate of lambda is: %3.3f" % lmd)

The estimate of lambda is: 0.827


Set parameters for ML estimation.

In [8]:
est_params = {
    "beta": 0.95,
    "lambda": lmd,
    "x_start": np.array([0.1,0.01,1.0]),
    "euler": 0.57721,
    "states": num_states,
    "x": x,
    "bellman_emax": 1,
    "threshold": 1e-12,
}

Report the parameters of the model estimated using MLE with a nested fixed point algorithm:

1. Guess initial parameter values.

2. Solve dynamic programming problem.
3. Calculate the probability of replacement at each state.
4. Use model predictions (the probability calculated in the previous step) and data to compute the log-likelihood function.
5. Search over parameter values by repeating steps 2 − 4.

The objective function can be minimized using *scipy.optimize.minimize*.

In [9]:
# Partial in arguments, except fixed parameters
partial_llf = partial(
    rust_llf, 
    d=df[["d"]].to_numpy(),
    xt=df[["xt"]].to_numpy(),
    params=est_params, 
)    

In [10]:
# Minimize the function
result = opt.minimize(
    fun=partial_llf, x0=est_params["x_start"], method="L-BFGS-B"
)
print(f'{result.success = }')
print(result.message)
print("Number of MLE iterations: %2.0f" % result.nit)

result.success = True
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Number of MLE iterations: 19


Report the estimated parameters and the probability of replacement for every state: $P (1|x_t, \theta)$.

In [11]:
print("Est. parameters: %3.3f" % result.x[0], "%3.3f" % result.x[1], "%3.3f" % result.x[2])
print("True parameters: %3.3f" % theta[0], "%3.3f" % theta[1], "%3.3f" % theta[2])

if est_params["bellman_emax"]:
    EV,V01,numiter = bellman1(
        result.x,
        est_params,
)
else:
    EV,V01,numiter = bellman2(
    result.x,
    est_params,
)

prd1 = np.exp(V01[x, 1]) / (np.exp(V01[x, 0]) + np.exp(V01[x, 1]))

print(" ")
print("Choice-specific value function:")
print(V01)
print(" ")
print("Conditional choice probabilities:")
print(prd1.reshape(num_states, 1))

Est. parameters: 0.364 0.083 1.994
True parameters: 0.500 0.040 2.000
 
Choice-specific value function:
[[ -0.07900218  -2.07316209]
 [ -1.14050909  -2.07316209]
 [ -2.13662307  -2.07316209]
 [ -3.1199416   -2.07316209]
 [ -4.15770059  -2.07316209]
 [ -5.30468086  -2.07316209]
 [ -6.5941457   -2.07316209]
 [ -8.04172817  -2.07316209]
 [ -9.65325021  -2.07316209]
 [-11.43046578  -2.07316209]
 [-13.37380054  -2.07316209]]
 
Conditional choice probabilities:
[[0.11981746]
 [0.28238679]
 [0.51585992]
 [0.740156  ]
 [0.88939129]
 [0.96200331]
 [0.98923875]
 [0.99744862]
 [0.99948974]
 [0.99991367]
 [0.99998764]]
